In [1]:
import pandas as pd
import numpy as np
import os

# Umbrales físicos aceptables
rango_valido = {
    'GHI': (0, 1200),
    'DNI': (0, 1000),
    'DHI': (0, 600),
    'Tdry': (-40, 60),
    'Tdew': (-40, 60),
    'RH': (0, 100),
    'Pres': (800, 1100),
    'Wspd': (0, 60),
    'Wdir': (0, 360),
    'Snow Depth': (0, 200)
}

archivos = [
    'calama_corrupted.csv',
    'salvador_corrupted.csv',
    'Vallenar_corrupted.csv'
]

def limpiar_archivo(nombre_archivo):
    print(f"🔍 Procesando: {nombre_archivo}")
    
    df = pd.read_csv(nombre_archivo)
    original_cols = df.columns.tolist()

    # Limpiar filas con número incorrecto de columnas (opcional)
    df = df[df.columns[:15]]

    # Eliminar filas completamente vacías
    df.dropna(how='all', inplace=True)

    # Reemplazar valores fuera de rango por NaN y luego interpolar
    for col, (vmin, vmax) in rango_valido.items():
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df.loc[(df[col] < vmin) | (df[col] > vmax), col] = np.nan
            df[col] = df[col].interpolate(limit=3)
            df[col] = df[col].ffill().bfill()

    # Guardar archivo limpio
    nombre_limpio = nombre_archivo.replace("_corrupted", "_clean")
    df.to_csv(nombre_limpio, index=False)
    print(f"✅ Guardado: {nombre_limpio} ({len(df)} filas)\n")

# Aplicar a todos los archivos
for archivo in archivos:
    if os.path.exists(archivo):
        limpiar_archivo(archivo)
    else:
        print(f"❌ No se encontró el archivo: {archivo}")


🔍 Procesando: calama_corrupted.csv
✅ Guardado: calama_clean.csv (8760 filas)

🔍 Procesando: salvador_corrupted.csv
✅ Guardado: salvador_clean.csv (8760 filas)

🔍 Procesando: Vallenar_corrupted.csv
✅ Guardado: Vallenar_clean.csv (8760 filas)

